In this notebook we compute the association between brain transcriptomics' and the pseudo R2 maps of each subtype 

In [1]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
import libpysal
from spreg import ML_Lag
from spreg import t_stat
from statsmodels.stats.multitest import multipletests

/home/javi/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


Function to compute the association, using ML_LAG algorithm

In [2]:
def calcPvals(f, gen_mat, dist):
    p_vals = []
    t_vals = []
    for single_gen_exp in gen_mat:
        w = libpysal.weights.KNN(dist, 1)
        w = libpysal.weights.insert_diagonal(w, np.zeros(w.n))

        ml_lag_results = ML_Lag(
            single_gen_exp[:, None], f[:, None], w, name_y="Gene", name_x=["F"]
        )
        p_vals.append(t_stat(ml_lag_results)[1][1])
        t_vals.append(t_stat(ml_lag_results)[1][0])
    return p_vals, t_vals

Load the transcriptomic expression of each brain gene, the geodesic distance matrix of each region from Desikan-Killiany atlas and the pseudo R2 maps

In [3]:
gen_exp_mat = pd.read_csv(os.path.join("../data", "gen_exp_in_LH_SP70.csv"))
gen_labels = gen_exp_mat["GeneLabel"].values
gen_exp = gen_exp_mat.drop(columns=["GeneLabel"])
desikan_geo_dist = pd.read_csv(os.path.join("../data", "desikan_LH_geo_dist.csv"))
f_mdmr = pd.read_csv(os.path.join("../results/desikan", "mdmr_rsquare_euclidean_desikan_LH.csv"))
subtype_name = f_mdmr.columns

For each subtype, we generate a table including only the p-FDR significant genes and the t-statistic and p-FDR of each one

In [4]:
for idx, f_sub in enumerate(f_mdmr.values.T):
    p, t = calcPvals(f_sub, gen_exp.values, desikan_geo_dist)
    pfdr = multipletests(p, alpha=0.05, method="fdr_bh", is_sorted=False)

    gen_signif_mask = np.where(pfdr[1] < 0.05)
    gen_signif_label = gen_labels[gen_signif_mask]
    gen_signif_t = np.array(t)[gen_signif_mask]
    gen_signif_pfdr = pfdr[1][gen_signif_mask]

    df = pd.DataFrame(
        {
            "GeneLabel": gen_signif_label,
            "t-statistic": gen_signif_t,
            "p-FDR": gen_signif_pfdr,
        },
    )
    df.to_csv(os.path.join("../results/desikan", 
                           subtype_name[idx] + "_r2_gen_association_results.csv"), index=False)

/home/javi/anaconda3/lib/python3.8/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 13 disconnected components.
  warnings.warn(message)
/home/javi/anaconda3/lib/python3.8/site-packages/libpysal/weights/util.py:850: UserWarning: This function is deprecated. Use fill_diagonal instead.
  warn("This function is deprecated. Use fill_diagonal instead.")
/home/javi/anaconda3/lib/python3.8/site-packages/scipy/optimize/_minimize.py:783: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "


In [5]:
print("Subtype 1, # positive genes =", 
      sum(pd.read_csv("../results/desikan/Subtype_1_r2_gen_association_results.csv").iloc[:, 1]<0))
print("Subtype 1, # negative genes =", 
      sum(pd.read_csv("../results/desikan/Subtype_1_r2_gen_association_results.csv").iloc[:, 1]>0))

Subtype 1, # positive genes = 195
Subtype 1, # negative genes = 364


In [6]:
print("Subtype 2, # positive genes =", 
      sum(pd.read_csv("../results/desikan/Subtype_2_r2_gen_association_results.csv").iloc[:, 1]<0))
print("Subtype 2, # positive genes =", 
      sum(pd.read_csv("../results/desikan/Subtype_2_r2_gen_association_results.csv").iloc[:, 1]>0))

Subtype 2, # positive genes = 142
Subtype 2, # positive genes = 180
